In [51]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Image analysis
from skimage.transform import resize
from skimage.io import imread

import os  # Allows for python to move folders within the Operating System
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import pickle

In [3]:
# Loading Images into Console
datadir = 'Images/'
categories = ['ChickenHead', 'ElephantHead']

flat_data_arr = []
target_arr = []

for i in categories:
    print('loading... category: ', i)
    path = os.path.join(datadir, i)
    #print(path)
    for img in os.listdir(path):
        #print(img)
        img_array = imread(os.path.join(path, img))
        img_resized = resize(img_array, (150, 150, 3))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(categories.index(i))
    print("loaded category ", i, "sucessfully\n")

loading... category:  ChickenHead
loaded category  ChickenHead sucessfully

loading... category:  ElephantHead
loaded category  ElephantHead sucessfully



In [4]:
# Converting images into numbers
flat_data = np.array(flat_data_arr)
target = np.array(target_arr)

df = pd.DataFrame(flat_data)
df['Target'] = target

df.head()

,0,1,2,3,4,5,6,7,8,9,...,67491,67492,67493,67494,67495,67496,67497,67498,67499,Target
0,0.258824,0.258824,0.219608,0.231373,0.231373,0.192157,0.168627,0.168627,0.129412,0.180392,...,0.321569,0.321569,0.290196,0.301961,0.301961,0.270588,0.290196,0.290196,0.258824,0
1,0.121569,0.227451,0.105882,0.129412,0.227451,0.113725,0.176471,0.254902,0.149020,0.219608,...,0.784314,0.670588,0.513725,0.772549,0.658824,0.501961,0.768627,0.654902,0.498039,0
2,0.415686,0.698039,0.258824,0.384314,0.666667,0.235294,0.349020,0.619608,0.203922,0.321569,...,0.423529,0.615686,0.266667,0.435294,0.627451,0.270588,0.380392,0.572549,0.215686,0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,0.784314,0.729412,0.623529,0.776471,0.721569,0.615686,0.737255,0.682353,0.576471,0.690196,...,0.717647,0.643137,0.517647,0.725490,0.650980,0.525490,0.721569,0.647059,0.521569,0


In [5]:
# Data Selection for Analysis
X = df.values[:, :-1]
Y = df.values[:, -1]

# Data Splitting 
xtrain, xtest, ytrain, ytest, = train_test_split(X, Y, test_size=0.2, random_state=47)

In [45]:
# Loading Test Images into Console
testdir = 'testImages/'

flat_data_load = 0
test_arr = []

print('loading... category: ', testdir)
path = os.path.join(testdir)
#print(path)
for img in os.listdir(path):
    print(img)
    img_array = imread(os.path.join(path, img))
    img_resized = resize(img_array, (150, 150, 3))
    flat_data_load = img_resized.flatten()
    test_arr.append(flat_data_load.reshape(1, -1))
    #print(test_arr[-1])
print("loaded category ", testdir, "sucessfully\n")

loading... category:  testImages/
chicken1.png
chicken2.jpg
chicken3.jpg
elephent1.jpg
elephent2.jpg
loaded category  testImages/ sucessfully



__Random Forest__

In [60]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
param_grid = {'n_estimators':[20, 30, 40, 50, 60]}

model = GridSearchCV(rfc, param_grid)
model.fit(xtrain, ytrain)
print(model.best_params_)

{'n_estimators': 50}


In [61]:
param_grid = {'n_estimators':[50]}

model = GridSearchCV(rfc, param_grid)
model.fit(xtrain, ytrain)

ypred_train = model.predict(xtrain)
ypred_test = model.predict(xtest)
print('Train Accuracy: ', accuracy_score(ypred_train, ytrain))
print('Test Accuracy: ', accuracy_score(ypred_test, ytest))

Train Accuracy:  1.0
Test Accuracy:  0.925


In [62]:
pickle.dump(model, open('rfcModel.pkl', 'wb'))

In [63]:
model = pickle.load(open('rfcModel.pkl', 'rb'))

In [64]:
testImgNum = 1
for img in test_arr:
    p = model.predict(img)
    if p==0: print('Image', testImgNum, 'Prediction: ChickenHead')
    else: print('Image', testImgNum, 'Prediction: ElephantHead')
    testImgNum += 1

Image 1 Prediction: ChickenHead
Image 2 Prediction: ChickenHead
Image 3 Prediction: ChickenHead
Image 4 Prediction: ElephantHead
Image 5 Prediction: ElephantHead


__SVM (Support Vector Machine)__

In [12]:
from sklearn.svm import SVC
svc = SVC()
param_grid = {'C':[0.1, 1, 10, 100], 'gamma':[0.001, 0.01, 0.1, 1], 'kernel': ['rbf', 'poly']}

model = GridSearchCV(svc, param_grid)
model.fit(xtrain, ytrain)
print(model.best_params_)

{'C': 0.1, 'gamma': 0.001, 'kernel': 'poly'}


In [13]:
param_grid = {'C':[0.1], 'gamma':[0.001], 'kernel': ['poly']}

model = GridSearchCV(svc, param_grid)
model.fit(xtrain, ytrain)

ypred_train = model.predict(xtrain)
ypred_test = model.predict(xtest)
print('Train Accuracy: ', accuracy_score(ypred_train, ytrain))
print('Test Accuracy: ', accuracy_score(ypred_test, ytest))

Train Accuracy:  1.0
Test Accuracy:  0.95


In [17]:
pickle.dump(model, open('svmModel.pkl', 'wb'))

In [18]:
model = pickle.load(open('svmModel.pkl', 'rb'))

In [44]:
testImgNum = 1
for img in test_arr:
    p = model.predict(img)
    if p==0: print('Image', testImgNum, 'Prediction: ChickenHead')
    else: print('Image', testImgNum, 'Prediction: ElephantHead')
    testImgNum += 1

Image 1 Prediction: ElephantHead
Image 2 Prediction: ChickenHead
Image 3 Prediction: ChickenHead
Image 4 Prediction: ElephantHead
Image 5 Prediction: ElephantHead


__KNN__

In [52]:
from sklearn.neighbors import KNeighborsClassifier
param_grid = {'n_neighbors': [1, 2, 3, 4, 5]}
knn = KNeighborsClassifier()

model = GridSearchCV(knn, param_grid)
model.fit(xtrain, ytrain)
print(model.best_params_)

{'n_neighbors': 4}


In [56]:
param_grid = {'n_neighbors': [4]}

model = GridSearchCV(knn, param_grid)
model.fit(xtrain, ytrain)

ypred_train = model.predict(xtrain)
ypred_test = model.predict(xtest)
print('Train Accuracy: ', accuracy_score(ypred_train, ytrain))
print('Test Accuracy: ', accuracy_score(ypred_test, ytest))

Train Accuracy:  0.9625
Test Accuracy:  0.875


In [57]:
pickle.dump(model, open('knnModel.pkl', 'wb'))

In [58]:
model = pickle.load(open('knnModel.pkl', 'rb'))

In [59]:
testImgNum = 1
for img in test_arr:
    p = model.predict(img)
    if p==0: print('Image', testImgNum, 'Prediction: ChickenHead')
    else: print('Image', testImgNum, 'Prediction: ElephantHead')
    testImgNum += 1

Image 1 Prediction: ChickenHead
Image 2 Prediction: ChickenHead
Image 3 Prediction: ChickenHead
Image 4 Prediction: ElephantHead
Image 5 Prediction: ChickenHead


__Neural Network__

In [78]:
from sklearn.neural_network import MLPClassifier
nnet = MLPClassifier()
param_grid = {'hidden_layer_sizes': [1, 2, 3], 'activation': ['relu', 'tanh'], 'learning_rate': ['constant', 'adaptive']}

model = GridSearchCV(nnet, param_grid)
model.fit(xtrain, ytrain)
print(model.best_params_)

{'activation': 'tanh', 'hidden_layer_sizes': 3, 'learning_rate': 'adaptive'}


In [79]:
param_grid = {'hidden_layer_sizes': [3], 'activation': ['tanh'], 'learning_rate': ['adaptive']}

model = GridSearchCV(nnet, param_grid)
model.fit(xtrain, ytrain)

ypred_train = model.predict(xtrain)
ypred_test = model.predict(xtest)
print('Train Accuracy: ', accuracy_score(ypred_train, ytrain))
print('Test Accuracy: ', accuracy_score(ypred_test, ytest))

Train Accuracy:  0.475
Test Accuracy:  0.6


In [80]:
pickle.dump(model, open('neuralnetworkModel.pkl', 'wb'))

In [81]:
model = pickle.load(open('neuralnetworkModel.pkl', 'rb'))

In [82]:
testImgNum = 1
for img in test_arr:
    p = model.predict(img)
    if p==0: print('Image', testImgNum, 'Prediction: ChickenHead')
    else: print('Image', testImgNum, 'Prediction: ElephantHead')
    testImgNum += 1

Image 1 Prediction: ChickenHead
Image 2 Prediction: ChickenHead
Image 3 Prediction: ChickenHead
Image 4 Prediction: ChickenHead
Image 5 Prediction: ChickenHead


__Adaboost__

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier()
param_grid = {'n_estimators': [1, 2, 5, 8, 10], 'learning_rate': [0.1, 0.001, 0.0001, 0]}

model = GridSearchCV(abc, param_grid)
model.fit(xtrain, ytrain)
print(model.best_params_)

In [ ]:
param_grid = {'n_estimators': [], 'learning_rate': []}

model = GridSearchCV(abc, param_grid)
model.fit(xtrain, ytrain)

ypred_train = model.predict(xtrain)
ypred_test = model.predict(xtest)
print('Train Accuracy: ', accuracy_score(ypred_train, ytrain))
print('Test Accuracy: ', accuracy_score(ypred_test, ytest))

In [ ]:
pickle.dump(model, open('abcModel.pkl', 'wb'))

In [ ]:
model = pickle.load(open('abcModel.pkl', 'rb'))

In [ ]:
testImgNum = 1
for img in test_arr:
    p = model.predict(img)
    if p==0: print('Image', testImgNum, 'Prediction: ChickenHead')
    else: print('Image', testImgNum, 'Prediction: ElephantHead')
    testImgNum += 1